In [1]:
import pandas as pd
import numpy as np
import time,datetime

In [2]:
def datetime2SecondsFrom1970(timeDateStr:str):
    time1=datetime.datetime.strptime(timeDateStr,"%Y-%m-%d %H:%M:%S")
    secondsFrom1970=time.mktime(time1.timetuple())
    return secondsFrom1970

def seconds2Datetime(seconds_from_1970):
    timeArray = time.localtime(seconds_from_1970)#1970秒数
    otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
    datetime1=datetime.datetime.strptime(otherStyleTime, "%Y-%m-%d %H:%M:%S")
    return str(datetime1)
def datetime2SecondsFrom1970_s(timeDateStr:str):
    time1=datetime.datetime.strptime(timeDateStr,"%Y%m%d%H%M%S")
    secondsFrom1970=time.mktime(time1.timetuple())
    return int(secondsFrom1970)

In [ ]:
ad_sample_test=pd.read_csv('../Data/A_preliminary_generate/Test/ad_sample_feature2phase_test.csv')

In [3]:
ad_static_feature=pd.read_csv("../Data/A_preliminary/testA/ad_static_feature.out",header=None,sep='\t',
                             names=['ad_id','create_time','ad_account_id','commodity_id','commodity_type','ad_trades_id','ad_size',
                             ])
train_set_3_19=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0319_feature1phase_train.csv')
valid_set_from_3_19=train_set_3_19[:2000]#设定验证集
train_set_3_19=train_set_3_19[2000:]
train_set_3_18=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0318_feature1phase_train.csv')
train_set=pd.concat([train_set_3_19,train_set_3_18],axis=0)
train_set['ad_trades_id']=train_set['ad_trades_id'].apply(lambda x: int(str(x).split(',')[0]))
#3月18,19日两天有275条样本交易行业超过一个，但是test_sample中不存在这样的样本，直接取第一个
for day in ['0317','0316','0315','0314','0313','0312','0311','0310','0309','0308','0307','0306','0305','0304']:
    train_set_date=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_'+day+'_feature1phase_train.csv')
    train_set=pd.concat([train_set,train_set_date],axis=0)
need_round_list=['ad_id_bid_mean','ad_id_bid_var','ad_id_pctr_mean','ad_id_pctr_var','ad_id_quality_ecpm_mean',
'ad_id_quality_ecpm_var','ad_id_total_ecpm_mean','ad_id_total_ecpm_var', 
'account_id_bid_mean','account_id_bid_var','account_id_pctr_mean','account_id_pctr_var',
'account_id_quality_ecpm_mean','account_id_quality_ecpm_var','account_id_total_ecpm_mean','account_id_total_ecpm_var',
'commodity_id_bid_mean','commodity_id_bid_var','commodity_id_pctr_mean','commodity_id_pctr_var',
'commodity_id_quality_ecpm_mean','commodity_id_quality_ecpm_var','commodity_id_total_ecpm_mean','commodity_id_total_ecpm_var',
'commodity_type_bid_mean','commodity_type_bid_var','commodity_type_pctr_mean',
'commodity_type_pctr_var','commodity_type_quality_ecpm_mean','commodity_type_quality_ecpm_var','commodity_type_total_ecpm_mean',
'commodity_type_total_ecpm_var','trades_id_bid_mean','trades_id_bid_var','trades_id_pctr_mean','trades_id_pctr_var',
'trades_id_quality_ecpm_mean','trades_id_quality_ecpm_var','trades_id_total_ecpm_mean','trades_id_total_ecpm_var',]
for col in need_round_list:
    train_set[col]=train_set[col].apply(lambda x:round(x,2))
train_set.drop(columns='create_time',inplace=True)
valid_set_from_3_19.drop(columns='create_time',inplace=True)
print('train\n',train_set.info())
train_set=train_set.merge(ad_static_feature[['ad_id','create_time']],on='ad_id',how='inner')
valid_set_from_3_19=valid_set_from_3_19.merge(ad_static_feature[['ad_id','create_time']],on='ad_id',how='inner')
train_set['have_create_seconds']=train_set['end_time']-train_set['create_time']
valid_set_from_3_19['have_create_seconds']=valid_set_from_3_19['end_time']-valid_set_from_3_19['create_time']
print('train\n',train_set.info())
print('valid\n',valid_set_from_3_19.info())
print(train_set.shape[0])
train_set.drop_duplicates(inplace=True)
print(train_set.shape[0])#去重
train_set=train_set[(train_set['hold_time']>10800)]#持续时间需要超过三个小时的样本
# train_set=train_set[(train_set['audience_orientation_nums']>0)]#定向人群数量为0的是有问题的
train_set.reset_index(inplace=True)
train_set.info()
train_set_phase1=train_set

/home/zhangyier/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 236611 entries, 2000 to 18527
Columns: 108 entries, ad_id to week_day
dtypes: float64(46), int64(61), object(1)
memory usage: 196.8+ MB
train
 None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 236611 entries, 0 to 236610
Columns: 110 entries, ad_id to have_create_seconds
dtypes: float64(46), int64(63), object(1)
memory usage: 200.4+ MB
train
 None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Columns: 110 entries, ad_id to have_create_seconds
dtypes: float64(46), int64(63), object(1)
memory usage: 1.7+ MB
valid
 None
236611
236611
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228871 entries, 0 to 228870
Columns: 111 entries, index to have_create_seconds
dtypes: float64(46), int64(64), object(1)
memory usage: 193.8+ MB


In [4]:
def check(train_set,train_set_phase1):
    columns_list=['ad_id','bid','audience_orientation_nums',
    'ad_id_history_exposure_times','ad_id_bid_mean','ad_id_bid_var','ad_id_pctr_mean','ad_id_pctr_var','ad_id_quality_ecpm_mean',
    'ad_id_quality_ecpm_var','ad_id_total_ecpm_mean','ad_id_total_ecpm_var', 
    'account_id_history_exposure_times','account_id_bid_mean','account_id_bid_var','account_id_pctr_mean','account_id_pctr_var',
    'account_id_quality_ecpm_mean','account_id_quality_ecpm_var','account_id_total_ecpm_mean','account_id_total_ecpm_var',
    'commodity_id_history_exposure_times','commodity_id_bid_mean','commodity_id_bid_var','commodity_id_pctr_mean','commodity_id_pctr_var',
    'commodity_id_quality_ecpm_mean','commodity_id_quality_ecpm_var','commodity_id_total_ecpm_mean','commodity_id_total_ecpm_var',
    'commodity_type_history_exposure_times','commodity_type_bid_mean','commodity_type_bid_var','commodity_type_pctr_mean',
    'commodity_type_pctr_var','commodity_type_quality_ecpm_mean','commodity_type_quality_ecpm_var','commodity_type_total_ecpm_mean',
    'commodity_type_total_ecpm_var',
    'trades_id_history_exposure_times','trades_id_bid_mean','trades_id_bid_var','trades_id_pctr_mean','trades_id_pctr_var',
    'trades_id_quality_ecpm_mean','trades_id_quality_ecpm_var','trades_id_total_ecpm_mean','trades_id_total_ecpm_var',
    'week_day',
    ]
    for col in columns_list:
        print(col,len(set(train_set[col]))-len(set(train_set[col])&set(train_set_phase1[col])))
        print(col,len(set(train_set_phase1[col]))-len(set(train_set[col])&set(train_set_phase1[col])))

In [14]:
train_set.sort_values(by=['ad_id','bid'],inplace=True)
train_set_phase1.sort_values(by='ad_id',inplace=True)
# columns_list=['ad_id','bid','audience_orientation_nums',
#     'ad_id_history_exposure_times','ad_id_bid_mean','ad_id_bid_var','ad_id_pctr_mean','ad_id_pctr_var','ad_id_quality_ecpm_mean',
#     'ad_id_quality_ecpm_var','ad_id_total_ecpm_mean','ad_id_total_ecpm_var', 
#     'account_id_history_exposure_times','account_id_bid_mean','account_id_bid_var','account_id_pctr_mean','account_id_pctr_var',
#     'account_id_quality_ecpm_mean','account_id_quality_ecpm_var','account_id_total_ecpm_mean','account_id_total_ecpm_var',
#     'commodity_id_history_exposure_times','commodity_id_bid_mean','commodity_id_bid_var','commodity_id_pctr_mean','commodity_id_pctr_var',
#     'commodity_id_quality_ecpm_mean','commodity_id_quality_ecpm_var','commodity_id_total_ecpm_mean','commodity_id_total_ecpm_var',
#     'commodity_type_history_exposure_times','commodity_type_bid_mean','commodity_type_bid_var','commodity_type_pctr_mean',
#     'commodity_type_pctr_var','commodity_type_quality_ecpm_mean','commodity_type_quality_ecpm_var','commodity_type_total_ecpm_mean',
#     'commodity_type_total_ecpm_var',
#     'trades_id_history_exposure_times','trades_id_bid_mean','trades_id_bid_var','trades_id_pctr_mean','trades_id_pctr_var',
#     'trades_id_quality_ecpm_mean','trades_id_quality_ecpm_var','trades_id_total_ecpm_mean','trades_id_total_ecpm_var',
#     'week_day',
#     ]
# for col in columns_list:
#     train_set[col]=train_set[col].apply(lambda x :int(x))
#     train_set_phase1[col]=train_set_phase1[col].apply(lambda x :int(x))
(train_set[columns_list].values==train_set_phase1[columns_list].values).all()

False

In [16]:
print(train_set.head(5))
print(train_set_phase1.head(5))

        index  ad_id  bid audience_targeting  start_time    end_time  \
161997  14143     32   69          area:7572  1551907895  1551994295   
176867  13558     32   69          area:7572  1551907895  1551994295   
41651   10209     32   70          area:7572  1552685501  1552771892   
30535   10707     32   70          area:7572  1552685501  1552771892   
57830   15573     32   72          area:7572  1552599096  1552685501   

        ad_account_id  commodity_id  commodity_type  ad_trades_id  \
161997          18752         32534              13           136   
176867          18752         32534              13           136   
41651           18752         32534              13           136   
30535           18752         32534              13           136   
57830           18752         32534              13           136   

                ...            device_item_nums  work_item_nums  \
161997          ...                           0               0   
176867          ..

In [8]:
check(train_set,train_set_phase1)

ad_id 0
ad_id 0
bid 0
bid 0
audience_orientation_nums 0
audience_orientation_nums 0
ad_id_history_exposure_times 0
ad_id_history_exposure_times 0
ad_id_bid_mean 0
ad_id_bid_mean 0
ad_id_bid_var 0
ad_id_bid_var 0
ad_id_pctr_mean 0
ad_id_pctr_mean 0
ad_id_pctr_var 0
ad_id_pctr_var 0
ad_id_quality_ecpm_mean 0
ad_id_quality_ecpm_mean 0
ad_id_quality_ecpm_var 0
ad_id_quality_ecpm_var 0
ad_id_total_ecpm_mean 0
ad_id_total_ecpm_mean 0
ad_id_total_ecpm_var 0
ad_id_total_ecpm_var 0
account_id_history_exposure_times 0
account_id_history_exposure_times 0
account_id_bid_mean 0
account_id_bid_mean 0
account_id_bid_var 0
account_id_bid_var 0
account_id_pctr_mean 0
account_id_pctr_mean 0
account_id_pctr_var 0
account_id_pctr_var 0
account_id_quality_ecpm_mean 0
account_id_quality_ecpm_mean 0
account_id_quality_ecpm_var 0
account_id_quality_ecpm_var 0
account_id_total_ecpm_mean 0
account_id_total_ecpm_mean 0
account_id_total_ecpm_var 0
account_id_total_ecpm_var 0
commodity_id_history_exposure_times 0


In [5]:
train_set_3_19=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0319_feature2phase_train.csv')
valid_set_from_3_19=train_set_3_19[:2000]#设定验证集
train_set_3_19=train_set_3_19[2000:]
train_set_3_18=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0318_feature2phase_train.csv')
train_set_3_17=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0317_feature2phase_train.csv')
train_set_3_16=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0316_feature2phase_train.csv')
train_set_3_15=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0315_feature2phase_train.csv')
train_set_3_14=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0314_feature2phase_train.csv')
train_set_3_13=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0313_feature2phase_train.csv')
train_set_3_12=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0312_feature2phase_train.csv')
train_set_3_11=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0311_feature2phase_train.csv')
train_set_3_10=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0310_feature2phase_train.csv')
train_set_3_09=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0309_feature2phase_train.csv')
train_set_3_08=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0308_feature2phase_train.csv')
train_set_3_07=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0307_feature2phase_train.csv')
train_set_3_06=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0306_feature2phase_train.csv')
train_set_3_05=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0305_feature2phase_train.csv')
train_set_3_04=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0304_feature2phase_train.csv')
# train_set_3_03=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0303_feature1phase_train.csv')

In [6]:
train_set=pd.DataFrame([])
train_set=pd.concat([train_set,train_set_3_19],axis=0)
train_set=pd.concat([train_set,train_set_3_18],axis=0)
train_set['ad_trades_id']=train_set['ad_trades_id'].apply(lambda x: int(str(x).split(',')[0]))
#3月18,19日两天有275条样本交易行业超过一个，但是test_sample中不存在这样的样本，直接取第一个
train_set=pd.concat([train_set,train_set_3_17],axis=0)
train_set=pd.concat([train_set,train_set_3_16],axis=0)
train_set=pd.concat([train_set,train_set_3_15],axis=0)
train_set=pd.concat([train_set,train_set_3_14],axis=0)
train_set=pd.concat([train_set,train_set_3_13],axis=0)
train_set=pd.concat([train_set,train_set_3_12],axis=0)
train_set=pd.concat([train_set,train_set_3_11],axis=0)
train_set=pd.concat([train_set,train_set_3_10],axis=0)
train_set=pd.concat([train_set,train_set_3_09],axis=0)
train_set=pd.concat([train_set,train_set_3_08],axis=0)
train_set=pd.concat([train_set,train_set_3_07],axis=0)
train_set=pd.concat([train_set,train_set_3_06],axis=0)
train_set=pd.concat([train_set,train_set_3_05],axis=0)
train_set=pd.concat([train_set,train_set_3_04],axis=0)
# train_set=pd.concat([train_set,train_set_3_03],axis=0)
print('train\n',train_set.info())
print('valid\n',valid_set_from_3_19.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 236611 entries, 2000 to 18527
Columns: 129 entries, ad_id to users_watch_ad_median
dtypes: float64(53), int64(75), object(1)
memory usage: 234.7+ MB
train
 None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 129 entries, ad_id to users_watch_ad_median
dtypes: float64(53), int64(75), object(1)
memory usage: 2.0+ MB
valid
 None


In [7]:
print(train_set.shape[0])
train_set.drop_duplicates(inplace=True)
print(train_set.shape[0])#去重
train_set=train_set[(train_set['hold_time']>10800)]#持续时间需要超过三个小时的样本
# train_set=train_set[(train_set['audience_orientation_nums']>0)]#定向人群数量为0的是有问题的
train_set.reset_index(inplace=True)
train_set.info()

236611
236611
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228871 entries, 0 to 228870
Columns: 130 entries, index to users_watch_ad_median
dtypes: float64(53), int64(76), object(1)
memory usage: 227.0+ MB


In [ ]:
last_index=0
for index in range(5,train_set_3_04.shape[1]+10,5):
    print(list(train_set_3_04.columns)[last_index:index])
    last_index=index

In [ ]:
print(set(train_set_3_04.columns)-set(columns_list))
print(set(ad_sample_test.columns)-set(columns_list))
print(len(columns_list),train_set_3_04.shape[1],ad_sample_test.shape[1])

## validation_set split  &  feature fixed

In [ ]:
columns_list=[
# 'ad_id','audience_targeting','end_time','ad_account_id','commodity_id','commodity_type','ad_trades_id','ad_size','hold_time',
#  'create_time','sample_id', 'exposure_ad_id',
 'bid','audience_orientation_nums','0000_0030','0030_0100','0100_0130','0130_0200','0200_0230','0230_0300','0300_0330',
'0330_0400','0400_0430','0430_0500','0500_0530','0530_0600','0600_0630','0630_0700','0700_0730','0730_0800','0800_0830','0830_0900',
'0900_0930','0930_1000','1000_1030','1030_1100','1100_1130','1130_1200','1200_1230','1230_1300','1300_1330','1330_1400','1400_1430','1430_1500',
'1500_1530','1530_1600','1600_1630','1630_1700','1700_1730','1730_1800','1800_1830','1830_1900','1900_1930','1930_2000','2000_2030','2030_2100',
'2100_2130','2130_2200','2200_2230','2230_2300','2300_2330','2330_2400',
'ad_id_history_exposure_times','ad_id_bid_mean','ad_id_bid_var','ad_id_pctr_mean','ad_id_pctr_var','ad_id_quality_ecpm_mean',
'ad_id_quality_ecpm_var','ad_id_total_ecpm_mean','ad_id_total_ecpm_var', 
'account_id_history_exposure_times','account_id_bid_mean','account_id_bid_var','account_id_pctr_mean','account_id_pctr_var',
'account_id_quality_ecpm_mean','account_id_quality_ecpm_var','account_id_total_ecpm_mean','account_id_total_ecpm_var',
'commodity_id_history_exposure_times','commodity_id_bid_mean','commodity_id_bid_var','commodity_id_pctr_mean','commodity_id_pctr_var',
'commodity_id_quality_ecpm_mean','commodity_id_quality_ecpm_var','commodity_id_total_ecpm_mean','commodity_id_total_ecpm_var',
'commodity_type_history_exposure_times','commodity_type_bid_mean','commodity_type_bid_var','commodity_type_pctr_mean',
'commodity_type_pctr_var','commodity_type_quality_ecpm_mean','commodity_type_quality_ecpm_var','commodity_type_total_ecpm_mean',
'commodity_type_total_ecpm_var',
'trades_id_history_exposure_times','trades_id_bid_mean','trades_id_bid_var','trades_id_pctr_mean','trades_id_pctr_var',
'trades_id_quality_ecpm_mean','trades_id_quality_ecpm_var','trades_id_total_ecpm_mean','trades_id_total_ecpm_var',
'week_day','have_create_seconds',
#feature phase 2
# 'have_create_days', 'history_exposure_times_mean', 'age_item_nums', 'gender_item_nums',
# 'area_item_nums', 'status_item_nums', 'education_item_nums', 'consuption_ability_item_nums', 'device_item_nums',
# 'work_item_nums', 'connection_type_item_nums', 'behavior_item_nums', 'day_before_exposure_times', 'users_watch_ad_sum',
# 'users_watch_ad_mean', 'users_watch_ad_var', 'users_watch_ad_max', 'users_watch_ad_median'
]
label=train_set['exposure_times']
train=train_set[columns_list]
test=ad_sample_test[columns_list]
label_validtion=valid_set_from_3_19['exposure_times']
validtion=valid_set_from_3_19[columns_list]

label_valid_array=label_validtion.values
validtion_array=validtion.values
train_array=train.values
label_array=label.values
test_array=test.values

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import KFold, RepeatedKFold
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split

In [ ]:
def smape(valid_,label_valid_array):
    temp_differ_abs=abs(valid_-label_valid_array)
    temp_sum_mod_2=(valid_+label_valid_array)/2
    
    print(40*(1-(sum(temp_differ_abs/temp_sum_mod_2)/label_valid_array.shape[0]/2)))
    return sum(temp_differ_abs/temp_sum_mod_2)/label_valid_array.shape[0]

In [ ]:
xgb_params = {'eta': 0.005, 'max_depth': 10, 'subsample': 0.8, 'colsample_bytree': 0.8,
          'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 10}
folds = KFold(n_splits=5, shuffle=True, random_state=2019)
oof_xgb = np.zeros(len(train_array))
predictions_xgb = np.zeros(len(test_array))
valid_xgb=np.zeros(len(label_valid_array))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_array, label_array)):
    print("fold n°{}".format(fold_ + 1))
    trn_data = xgb.DMatrix(train_array[trn_idx], label_array[trn_idx])
    val_data = xgb.DMatrix(train_array[val_idx], label_array[val_idx])

    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200,
                    verbose_eval=100, params=xgb_params)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(train_array[val_idx]), ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(test_array), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    valid_xgb += clf.predict(xgb.DMatrix(validtion_array), ntree_limit=clf.best_ntree_limit) / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, label_array)))
print("cv smape:",smape(oof_xgb, label_array))
print('validation:',mean_squared_error(valid_xgb, label_valid_array))
print(smape(valid_xgb,label_valid_array))

In [ ]:
test_sample=pd.read_csv("../Data/A_preliminary/testA/test_sample.dat",header=None,sep='\t',
                        names=['sample_id','ad_id','create_time','ad_size','ad_trades_id','commodity_type','commodity_id','ad_account_id',
                              'when_ad_put','audience_targeting','bid'])
print(test_sample.head())
submission=test_sample[['sample_id','ad_id','bid']]
submission.insert(submission.shape[1], 'predict_exposure', predictions_xgb)
# submission.insert(submission.shape[1], 'predict_exposure', predictions_lgb)

submission['predict_exposure']=submission['predict_exposure'].apply(lambda x : round(x,4))
ad_id_exposure_predic=submission.groupby(['ad_id'])['predict_exposure'].aggregate(['mean','max','min']).reset_index()
ad_id_dict={}
for ad_id,mean,min_,max_ in zip(ad_id_exposure_predic['ad_id'],ad_id_exposure_predic['mean'],ad_id_exposure_predic['min'],ad_id_exposure_predic['max']):
#     ad_id_dict[ad_id]=mean-(max_-min_)/8
    ad_id_dict[ad_id]=mean
#     ad_id_dict[ad_id]=max_

print('how many ad_id exposure less 1 times :')
nums=0
for _ in ad_id_dict.values():
    if _ < 1:
        nums+=1
print(nums)
print('submission_csv mean',submission_csv['explosion'].mean())

print('Monotonic adjust!!!')
ad_id_explosure_df=pd.DataFrame({"ad_id":list(ad_id_dict.keys()),"explosion_init":list(ad_id_dict.values())})
submission_csv=test_sample[['sample_id','ad_id','bid']][:]
submission_csv=submission_csv.merge(ad_id_explosure_df,on='ad_id',how='outer')
submission_csv['explosion']=submission_csv['explosion_init']+submission_csv['bid']/1000
submission_csv['explosion']=submission_csv['explosion'].apply(lambda x:round(x,4))
# submission_csv[['sample_id','explosion']].to_csv('submission.csv',header=None,index=None,encoding='utf-8')